1. +gsp_persons.birth_year
2. +gsp_persons.birth_region_id
3. +gsp_persons.living_region_id
4. +doesLive_birth_region ( 0,1 )
5. +gsp_persons.nationality_id
6. +gsp_persons.sex_id
7. +mib_orders 3 types -> ( jarayonda, tugallangan, yo'q )
8. -work (ishli yoki yo'q)
9. +zags 3 types -> (turmush qurgan, turmush qurmagan, ajrashgan, vagot etgan )
10. +ma'lumoti ( oliy, o'rta, unknown )
11. +Qarindoshlar orasida defendantlar bor yoki yo'qligi
(ota-ona, aka-uka-opa-singil, opa-singillarning-turmush-o'rtog'i, ota-onaning-aka-uka,opa-singillari) (3 labels: parents, siblings, siblings of parents, ---- husband of sisters, childrens of siblings of parents)
12. +mashinalari soni (YHXX.tech_passport)
13. +jinoyat ko'p sodir bo'ladigan joyga bog'liqligi
14. +isviolent(oldin zo'ravonlik qilgan yoki yo'qligi, agar pinppi iv_violence bo'lsa-1)
15. +administrative.pinpp (mamuriy ja'zo qo'llanganmi)
17. +bandlik pinpp check (searcher+worker,retraining listlardan birortasida bor yoki yo'qligi)
18. +agar bandlik tableda bo'lsa oyligini tekshirish(yo'q bo'lsa 0 yoki oylik miqdori)
19. +iiv.emi ga tushganmi (pinpp bor yoki yo'qligi)?
21. +isviolence bo'yicha yashash joylari sortlanadi
22. -Mashina oldi sottishini tekshirish(buy_count, sell_count)
23. +zags_Count, divorce_count
22. +Farzandlari soni

In [1]:
import pandas as pd
import numpy as np
import clickhouse_connect
import bisect
import random
from typing import List, Tuple, Optional

In [2]:
host = "play.clickhouse.com"
port = 443
user = "explorer"
password = ""

In [29]:
def get_function_simple(column_name, table_name):
    query = """SELECT {column_name}  FROM {table_name}"""
    
    client = clickhouse_connect.get_client(host=host, port=port, user=user, password=password)
    result = client.query(query)
    client.close()

    return result.result_rows

Getting all pinpp of defendants

In [3]:
def get_all_pinpp_of_defendant():
    query = """SELECT defendant_pinfl  FROM defendants
WHERE toDate(defendant_provision_date) >= '2022-03-31'
ORDER BY defendant_provision_date DESC
"""
    
    client = clickhouse_connect.get_client(host=host, port=port, user=user, password=password)
    result = client.query(query)
    client.close()

    return result.result_rows

Getting limited pinpp for training

In [5]:
defendants_pinpp_all = get_all_pinpp_of_defendant()
defendants_pinpp_limited = defendants_pinpp_all[:1000]

Getting normal humans pinpp for training

In [6]:
def get_pinpp_of_normal_humans(not_get_pinfl_list, limit=100000):
    excluded_pinpps = ", ".join([f"'{pinpp}'" for pinpp in not_get_pinfl_list])
    
    query = f"""
    SELECT OriginAirportID
    FROM ontime
    WHERE OriginAirportID NOT IN ({excluded_pinpps})
    LIMIT {limit}
    """

    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

In [8]:
# Getting normal humans pinpp
normal_pinpp_all = get_pinpp_of_normal_humans(not_get_pinfl_list=defendants_pinpp_all, limit=1000)

Getting main overall pinpp and defendant label dataframe

In [10]:
df_def = pd.DataFrame()
df_def["pinpp"] = defendants_pinpp_limited
df_def["defendant"] = 1

df_normal = pd.DataFrame()
df_normal["pinpp"] = normal_pinpp_all
df_normal["defendant"] = 0

df_main = pd.concat([df_def, df_normal], ignore_index=True)
df_main = df_main.sample(frac=1)

In [11]:
pinpp_list = df_main["pinpp"].tolist()

### GETTING datas by pinpp

In [14]:
def main_function_toget_by_pinpp(pinpp_list, column_name, table_name):
    pinpp_values = ', '.join(f"'{pinpp}'" for pinpp in pinpp_list)
    query = f"""
    SELECT {column_name}
    FROM {table_name}
    WHERE pinpp IN ({pinpp_values})
    """
    
    client = clickhouse_connect.get_client(
        host=host,
        port=port,
        user=user,
        password=password,
        secure=True 
    )
    
    result = client.query(query)
    client.close() 
    return result.result_rows

BIRTH YEAR -1

In [20]:
birth_year_list = main_function_toget_by_pinpp(pinpp_list, "birth_year", 'gsp_persons')

BIRTH_REGION_ID - 2

In [21]:
birth_region_id = main_function_toget_by_pinpp(pinpp_list, "birth_region_id", 'gsp_persons')

LIVING_REGION_ID -3

In [23]:
living_region_id = main_function_toget_by_pinpp(pinpp_list, 'living_region_id', 'gsp_persons')

DOES LIVE IN BIRTH REGION - 4

In [22]:
# doeslive_in_birth_region =[]
# for index in len(range(pinpp_list)):
#     if birth_region_id[index] == region_id[index]:
#         doeslive_in_birth_region.append(1)
#     else:
#         doeslive_in_birth_region.append(0)

Nationality id - 5

In [18]:
nationality_id = main_function_toget_by_pinpp(pinpp_list, "nationality_id", "gsp_persons")

SEX ID - 6

In [19]:
# sex_id = main_function_toget_by_pinpp(pinpp, "sex_id", "gsp_persons")

MIB ORDERS - 7

In [25]:
list_mib_pinpp = main_function_toget_by_pinpp(pinpp_list, "mib_orders",'pinpp')
list_mib_status = main_function_toget_by_pinpp(pinpp_list, "mib_orders",'doc_status_id')
final_list_miborder = []
for t in range(len(pinpp_list)):
    if pinpp_list[t] not in list_mib_pinpp:
        final_list_miborder.append(0)
    else:
        if list_mib_status[list_mib_pinpp.index(pinpp_list[t])] == 'yopilgan':
            final_list_miborder.append(1)
        else:
            final_list_miborder.append(2)

ZAGS - 9

In [26]:
result_list_zags = []

marriage_husband_pinpp = main_function_toget_by_pinpp(pinpp_list,"husband_pinpp", "zags_marriage")
marriage_wife_pinpp = main_function_toget_by_pinpp(pinpp_list,"wife_pinpp", "zags_marriage")
overall_marriage_pinpp = marriage_wife_pinpp + marriage_husband_pinpp

divorce_husband_pinpp = main_function_toget_by_pinpp(pinpp_list,"husband_pinpp", "zags_divorce")
divorce_wife_pinpp = main_function_toget_by_pinpp(pinpp_list,"wife_pinpp", "zags_divorce")
overall_divorce_pinpp = divorce_wife_pinpp + divorce_husband_pinpp

death_husband_pinpp = main_function_toget_by_pinpp(pinpp_list,"husband_pinpp", "zags_death")
death_wife_pinpp = main_function_toget_by_pinpp(pinpp_list,"wife_pinpp", "zags_death")
overall_death_pinpp = death_husband_pinpp + death_wife_pinpp

# 1-turmush qurgan, 2-turmush qurmagan, 3-ajrashgan, 4-vafot etgan

for index in pinpp_list:
    if index in overall_divorce_pinpp:
        result_list_zags.append(3)
    elif index in overall_death_pinpp:
        result_list_zags.append(4)
    elif index in overall_marriage_pinpp:
        result_list_zags.append(1)
    else:
        result_list_zags.append(2)

Is violent - 14

In [30]:
violents_list = get_function_simple('pinpp', "iiv_violence")

result_list_violents = []
for pinpp in pinpp_list:
    if pinpp in violents_list:
        result_list_violents.append(1)
    else:
        result_list_violents.append(0)


check administrative pinpp - 15

In [31]:
administrative_list = get_function_simple('pinpp', "administrative")

result_list_adms = []

for pinpp in pinpp_list:
    if pinpp in administrative_list:
        result_list_adms.append(1)
    else:
        result_list_adms.append(0)

bandlik checking - 17, salary - 18

In [33]:
bandlik_1 = get_function_simple('pinpp', "bandlik_1")
bandlik_2 = get_function_simple('pinpp', "bandlik_2")

salart_1 = get_function_simple('pinpp', "salart_1")
salary_2 = get_function_simple('pinpp', "salary_2")

overall_bandlik = bandlik_1+bandlik_2
overall_salary = salart_1+salary_2

result_list_bandlik = []
salary = []

for pinpp in pinpp_list:
    if pinpp in overall_bandlik:
        result_list_bandlik.append(1)
        salary.append(overall_salary[overall_bandlik.index(pinpp)])
    else:
        result_list_bandlik.append(0)

ivv_emi checking - 19

In [34]:
iiv_emi = get_function_simple('pinpp', "ii_emi")

iiv_emi_list = []

for pinpp in pinpp_list:
    if pinpp in iiv_emi:
        iiv_emi_list.append(1)
    else:
        iiv_emi_list.append(0)

Zags Count, Divorce Count - 23

In [35]:
zags_list = get_function_simple('pinpp', "zags_marriage")
divorce_list = get_function_simple('pinpp', "zags_divorce")

result_list_zags_count = []
result_list_divorce_count = []

for index_marriage in pinpp_list:
    result_list_zags_count.append(zags_list.count(index_marriage))

for index_divorce in pinpp_list:
    result_list_divorce_count.append(divorce_list.count(index_divorce))


Ma'lumoti - 10

In [1]:
def get_education_by_pinpp(pinpp_list):
    client = clickhouse_connect.get_client(host, port,user,password)

    pinpp_values = "', '".join(pinpp_list)  
    query = f"""
        SELECT 
            p.pinpp, 
            COALESCE(m.refEducationDegreeName, '0') AS education_degree
        FROM 
            (SELECT arrayJoin(['{pinpp_values}']) AS pinpp) AS p
        LEFT JOIN 
            mvd_statistics AS m
        ON 
            p.pinpp = m.pinpp
    """

    result = client.query(query).result_rows
    client.close()

    return result

In [2]:
list_pinpp_education = get_education_by_pinpp(pinpp_list)

Family checking - 11

In [4]:
def find_criminal_relations(pinpp_list, all_defendant):
    client = clickhouse_connect.get_client(host, port, user, password)

    results = {}
    for pinpp in pinpp_list:
        # 1. Ota-onalar PINPPlarini olish
        query_parents = f"""
            SELECT father_pinpp, mother_pinpp
            FROM gsp_persons
            WHERE pinpp = '{pinpp}'
        """
        parent_data = client.query(query_parents).result_rows
        
        if parent_data:
            father_pinpp, mother_pinpp = parent_data[0]
            father_criminal = 1 if father_pinpp in all_defendant else 0
            mother_criminal = 2 if mother_pinpp in all_defendant else 0
            parent_status = father_criminal + mother_criminal
        else:
            parent_status = 0

        # 2. Aka-uka/opa-singillarni olish
        query_siblings = f"""
            SELECT pinpp
            FROM gsp_persons
            WHERE father_pinpp = (SELECT father_pinpp FROM gsp_persons WHERE pinpp = '{pinpp}')
              AND mother_pinpp = (SELECT mother_pinpp FROM gsp_persons WHERE pinpp = '{pinpp}')
              AND pinpp != '{pinpp}'
        """
        siblings = [row[0] for row in client.query(query_siblings).result_rows]
        sibling_criminal = 1 if any(sibling in all_defendant for sibling in siblings) else 0

        # 3. Otaning aka-ukalari va opa-singillarini olish
        query_fathers_siblings = f"""
            SELECT pinpp
            FROM gsp_persons
            WHERE father_pinpp = (SELECT father_pinpp FROM gsp_persons WHERE pinpp = 
                                  (SELECT father_pinpp FROM gsp_persons WHERE pinpp = '{pinpp}'))
        """
        fathers_siblings = [row[0] for row in client.query(query_fathers_siblings).result_rows]
        fathers_sibling_criminal = 1 if any(relative in all_defendant for relative in fathers_siblings) else 0

        results[pinpp] = {
            "parent_status": parent_status,
            "sibling_criminal": sibling_criminal,
            "fathers_sibling_criminal": fathers_sibling_criminal
        }

    client.close()
    return results


Mashinalar soni - 12

In [5]:
tech_p = get_function_simple('pinpp', "tech_passport")
result_tech_p = []

for index in pinpp_list:
    result_tech_p.append(tech_p.count(index))


jinoyat ko'p sodir bo'ladigan joyga bog'liqligi

In [7]:
def label_pinpps_by_crime_region(pinpp_list):
    client = clickhouse_connect.get_client(host, port, user, password)
    
    query_defendants = """
    SELECT region_id, COUNT(*) as crime_count
    FROM defendants
    GROUP BY region_id
    ORDER BY crime_count DESC
    """
    crime_data = client.query(query_defendants).result_rows

    max_crime = max(crime_data, key=lambda x: x[1])[1]
    min_crime = min(crime_data, key=lambda x: x[1])[1]

    def normalize_crime_count(crime_count, max_crime, min_crime):
        return (crime_count - min_crime) / (max_crime - min_crime) * 10

    region_crime_rank = {region_id: normalize_crime_count(crime_count, max_crime, min_crime) 
                         for region_id, crime_count in crime_data}

    query_gsp_persons = f"""
    SELECT pinpp, region_id FROM gsp_persons
    WHERE pinpp IN ({','.join(map(str, pinpp_list))})
    """
    gsp_persons_data = client.query(query_gsp_persons).result_rows

    labelled_pinpps = [region_crime_rank[region_id] for pinpp, region_id in gsp_persons_data]

    return labelled_pinpps


In [8]:
labelled_pinpps = label_pinpps_by_crime_region(pinpp_list)
print(labelled_pinpps)

bandlik_pinpp_check (searcher+worker)

In [9]:
def check_pinpp_in_bandlik_tables(pinpp_list):
    client = clickhouse_connect.get_client(host,
                                           port,
                                           user,
                                           password)
    
    query_combined = f"""
    SELECT pinpp FROM defendants
    WHERE pinpp IN ({','.join(map(str, pinpp_list))})
    UNION
    SELECT pinpp FROM gsp_persons
    WHERE pinpp IN ({','.join(map(str, pinpp_list))})
    """
    combined_data = set(pinpp for pinpp, in client.query(query_combined).result_rows)

    result = [1 if pinpp in combined_data else 0 for pinpp in pinpp_list]

    return result

In [10]:
# result = check_pinpp_in_combined_tables(pinpp_list)
# print(result)

Is_violence sorting - 21

In [13]:
def label_pinpps_by_crime_and_violence(pinpp_list):
    client = clickhouse_connect.get_client(host,
                                           port,
                                           user,
                                           password)

    query_defendants = """
    SELECT region_id, COUNT(*) as crime_count
    FROM defendants
    GROUP BY region_id
    """
    query_violence = """
    SELECT destrict_id, COUNT(*) as violence_count
    FROM iiv_violence
    GROUP BY destrict_id
    """
    
    crime_data = client.query(query_defendants).result_rows
    violence_data = client.query(query_violence).result_rows

    combined_data = {}
    
    for region_id, crime_count in crime_data:
        combined_data[region_id] = {'crime': crime_count, 'violence': 0}
        
    for destrict_id, violence_count in violence_data:
        if destrict_id in combined_data:
            combined_data[destrict_id]['violence'] = violence_count
        else:
            combined_data[destrict_id] = {'crime': 0, 'violence': violence_count}

    max_crime = max(combined_data.values(), key=lambda x: x['crime'])['crime']
    min_crime = min(combined_data.values(), key=lambda x: x['crime'])['crime']
    
    max_violence = max(combined_data.values(), key=lambda x: x['violence'])['violence']
    min_violence = min(combined_data.values(), key=lambda x: x['violence'])['violence']

    def normalize(count, max_count, min_count):
        return (count - min_count) / (max_count - min_count) * 10

    for region_id in combined_data:
        crime = combined_data[region_id]['crime']
        violence = combined_data[region_id]['violence']
        combined_data[region_id]['crime_norm'] = normalize(crime, max_crime, min_crime)
        combined_data[region_id]['violence_norm'] = normalize(violence, max_violence, min_violence)

    query_gsp_persons = f"""
    SELECT pinpp, region_id FROM gsp_persons
    WHERE pinpp IN ({','.join(map(str, pinpp_list))})
    """
    gsp_persons_data = client.query(query_gsp_persons).result_rows

    labelled_pinpps = []
    for pinpp, region_id in gsp_persons_data:
        if region_id in combined_data:
            crime_norm = combined_data[region_id]['crime_norm']
            violence_norm = combined_data[region_id]['violence_norm']
            label = (crime_norm + violence_norm) / 2  
            labelled_pinpps.append(label)
        else:
            labelled_pinpps.append(0)  

    return labelled_pinpps

In [12]:
labelled_pinpps = label_pinpps_by_crime_and_violence(pinpp_list)
print(labelled_pinpps)

Farzandlari soni

In [15]:
def count_children(pinpp_list):
    client = clickhouse_connect.get_client(host,
                                           port,
                                           user,
                                           password)

    pinpp_list_str = ','.join([f"'{pinpp}'" for pinpp in pinpp_list])

    query = f"""
    SELECT
        p.pinpp,
        COUNT(b.father_pinpp) AS children_count
    FROM
        pinpp_list p
    LEFT JOIN
        zags_birth b ON b.father_pinpp = p.pinpp OR b.mother_pinpp = p.pinpp
    WHERE
        p.pinpp IN ({pinpp_list_str})
    GROUP BY
        p.pinpp
    """

    result = client.query(query).result_rows

    result_df = pd.DataFrame(result, columns=['pinpp', 'children_count'])

    return result_df

# result = count_children(pinpp_list)
# print(result)

In [16]:
from decimal import Decimal
Decimal('0.1') + Decimal('0.2')

Decimal('0.3')